In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import sqlite3

In [8]:
import sys
sys.path.append('../')


SyntaxError: invalid syntax (<ipython-input-8-51bdb8ea314f>, line 3)

# Repos

In [ ]:
import requests

base_url = "https://api.github.com"
user = "crawftv"

r = requests.get(f"{base_url}/users/{user}/repos?sort=pushed")


In [ ]:
from srv.data_types import RepoResponse
repo = RepoResponse(*list(r.json()[0].values()))

In [ ]:
# with sqlite3.connect("../portfolio.db") as c:
#     c.execute("""
#     CREATE TABLE repos 
#     (id int PRIMARY KEY, node_id text, name text UNIQUE, full_name text, private int, html_url text, description text, pushed_at_unix real, updated_at_unix real, created_at_unix real)""")

In [ ]:
for zz in range(1,6):
    r = requests.get(f"{base_url}/users/{user}/repos?page={zz}")
    for i in r.json():
        repo = RepoResponse(*i.values())
        with sqlite3.connect("portfolio.db") as c:
            repo_ids = [ i[0] for i in c.execute("Select id from repos")]
            try:
                c.execute(f"""INSERT INTO repos VALUES ({repo.id},'{repo.node_id}', "{repo.name}", "{repo.full_name}", {int(repo.private)},"{repo.html_url}","{repo.description}",{repo.pushed_at_unix},{repo.updated_at_unix},{repo.created_at_unix})""")
            except Exception as e:
                print(f"ERROR: {repo.name}, {e}")
        

In [ ]:
with sqlite3.connect("../portfolio.db") as c:
    cc = c.cursor().execute(f"""select count(id) from repos""")
    for row in cc:
        print(row)

In [6]:
# with sqlite3.connect("../portfolio.db") as c:
#     c.cursor().execute(f"""DROP VIEW repo_count; """)

# Events

In [5]:
import requests
from srv.data_types import EventResponse
base_url = "https://api.github.com"
user = "crawftv"

In [110]:
# with sqlite3.connect("../srv/portfolio.db") as c:
#     c.execute("""
#     CREATE TABLE events 
#     (id text PRIMARY KEY, type text, repo_id int, created_at_unix real, created_at_ymd text )
#     """)
#     c.execute("""
#     CREATE INDEX repo_event_idx on events(repo_id)""")

In [111]:
for zz in range(1,11):
    r = requests.get(f"{base_url}/users/{user}/events?page={zz}")
    for i in r.json():
        if i["type"] not in ["ForkEvent","WatchEvent"]:
            event = EventResponse(*i.values())
            with sqlite3.connect("../srv/portfolio.db") as c:
                c.execute(f"""
                INSERT INTO events
                VALUES ("{event.id}", "{event.type}",{event.repo_id}, {event.created_at_unix}, "{event.created_at_ymd}")
                """)

In [108]:
# with sqlite3.connect("../srv/portfolio.db") as c:
#     c.cursor().execute(f"""DROP TABLE events """)

In [112]:
with sqlite3.connect("../srv/portfolio.db") as c:
    cc = c.cursor().execute("SELECT count(*) from events")
    for i in cc:
        print(i)

(89,)


# Aggregate Events table

In [114]:
# with sqlite3.connect("../srv/portfolio.db") as c:
#     c.cursor().execute(f"""DROP TABLE aggregate_events """)

In [116]:
# ## create table
# with sqlite3.connect("file:../srv/portfolio.db",uri=True) as c:
#     cc = c.cursor().execute("""
#     CREATE TABLE aggregate_events
#     (created_at_ymd text PRIMARY KEY, event_count int DEFAULT 0, unix_time real)
#     """)

In [117]:
# get values from other tabel
with sqlite3.connect("file:../srv/portfolio.db?mode=ro",uri=True) as c:
    #c.row_factory = lambda c, r: dict(zip([col[0] for col in c.description], r))    
    cc = c.cursor().execute("""
    SELECT created_at_ymd , COUNT(created_at_ymd)
    FROM events
    GROUP BY created_at_ymd
    ORDER BY avg(created_at_unix)
    """)
    rows = [ i for i in  cc.fetchall()]

In [119]:
import dateutil
rows = [ (i[0],i[1], dateutil.parser.parse(i[0]).timestamp() ) for i in rows]

In [120]:
# insert the aggregated rows
with sqlite3.connect("file:../srv/portfolio.db",uri=True) as c:
    cc = c.cursor().executemany("""INSERT INTO aggregate_events VALUES (?,?,?)""", rows)    

In [121]:
#check it
with sqlite3.connect("file:../srv/portfolio.db?mode=ro",uri=True) as c:
    cc = c.cursor().execute("""
    SELECT * from aggregate_events
    ORDER BY unix_time
    """)
    rows = [i for i in cc.fetchall()]

In [123]:
def make_timestamp(date_string):
    return dateutil.parser.parse(date_string).timestamp()

In [128]:
# add missing values


# for i in range(18,32):
#     dstring = f"2020-01-{i:02d}"
#     row = (dstring,0,make_timestamp(dstring) )    
#     with sqlite3.connect("file:../srv/portfolio.db",uri=True) as c:
#         try : cc = c.cursor().execute("""
#         INSERT into aggregate_events VALUES(?,?,?)
#         """, row)
#         except Exception as e:
#             print(f"Error: {dstring}, {e}")